In [60]:
import numpy as np
import pandas as pd
import pickle
import xgboost
import time

In [61]:
import lightgbm as lgb
from xgboost.sklearn import XGBClassifier

warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import roc_curve, auc, accuracy_score, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve

In [62]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from collections import Counter, defaultdict
from random import choice
import os


### LOADING DATA

In [6]:
# sessions
with open (os.path.expanduser("~/USF/adv_ml/final/LeanPlum/session.pkl"), "rb") as f:
    sessions = pickle.load(f)

# event
with open (os.path.expanduser("~/USF/adv_ml/final/LeanPlum/events.pkl"), "rb") as f:
    events = pickle.load(f)

'''# choose certain columns    
session_df = sessions[["user_id_hash", "previous_sessions_duration"]]
event_df = events[["user_id_hash", "event_timestamp", "event", "event_value"]] '''

'# choose certain columns    \nsession_df = sessions[["user_id_hash", "previous_sessions_duration"]]\nevent_df = events[["user_id_hash", "event_timestamp", "event", "event_value"]] '

In [7]:
sessions.head()

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash
0,5558845121177764917,1542215364580,Asia/Manila,28800000.0,25837591,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,18781111175537580,1539215568666,Asia/Manila,28800000.0,11343848,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,1477540082628742048,1540120743010,Asia/Manila,28800000.0,13499724,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,8184875317380844086,1542671625528,Asia/Manila,28800000.0,32788010,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4706180700083856343,1538997913013,Asia/Manila,28800000.0,5872534,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,4,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [8]:
events.head()

,session_id,event,event_timestamp,event_value,user_id_hash
0,5558845121177764917,45,1542215397132,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,5558845121177764917,45,1542215484895,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,7689508378645584666,.m5100869650219008,1541124410372,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,2201961907282901522,4,1543713091129,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,2201961907282901522,6,1543713093116,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [9]:
sessions.shape

(5165759, 17)

In [10]:
events.shape

(111945519, 5)

## Labeling data 

Labels:
We divide the given dataset into training dataset and validation dataset by a cutoff date: Oct 14

In [11]:
# get unique user list 
users = pd.DataFrame(list(events.user_id_hash.unique()))
users.columns = ["user_id_hash"]

#### Use Dec-1 to Dec-7 and Dec-1 to Dec-14 to get labels for the validation set

In [12]:
# Validation labels

december_event = events[(events["event_timestamp"] >= 1543651199000) & #  >= Fri Nov 30 2018 23:59:59 GMT-0800
                  (events["event_timestamp"] < 1544255999000)] #  < Fri Dec 07 2018 23:59:59 GMT-0800

# Users who made a purchase in December
purchase_user = set(december_event[december_event["event"] == "8"].user_id_hash.unique()) 
# For validation, the users include all unique users
users_val = users

labels_dec = []
for user in list(events.user_id_hash.unique()):
    if user in purchase_user:
        labels_dec.append(1)
    else:
        labels_dec.append(0)
# label and add to val
users_val["purchased"] = pd.DataFrame(labels_dec)

In [13]:
users_val.shape

(620988, 2)

In [14]:
# second validation label

december_event_2 = events[(events["event_timestamp"] >= 1544255999000)] # Beyond 7 Dec 
purchase_user_2 = set(december_event_2[december_event_2["event"] == "8"].user_id_hash.unique())

labels_december_2 = []
for user in list(events.user_id_hash.unique()):
    if user in purchase_user_2:
        labels_december_2.append(1)
    else:
        labels_december_2.append(0)

users_val["purchased2"] = pd.DataFrame(labels_december_2)


In [63]:
users_val.head()

,user_id_hash,purchased,purchased2
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,1
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0


### Training labels

In [16]:
#  Use Oct-15 to Nov-30 to get features for users labeled in December
november_event = events[(events["event_timestamp"] >= 1542265200000)  # from Mon Nov 15 2018 00:00:00 GMT-0700
                          & (events["event_timestamp"] < 1542960000000)] # Fri Nov 23 2018 00:00:00 GMT-0800

purchase_user = set(november_event[november_event["event"] == "8"].user_id_hash.unique())
users_train = users
labels_november = []

for user in list(events.user_id_hash.unique()):
    if user in purchase_user:
        labels_november.append(1)
    else:
        labels_november.append(0)

users_train["purchased"] = pd.DataFrame(labels_november)

In [17]:
# second training label

november_event_2 = events[(events["event_timestamp"] >= 1542268800000) &  #from Thu Nov 15 2018 00:00:00 GMT-0800
                      (events["event_timestamp"] < 1543478400000)] # till Thu Nov 29 2018 00:00:00 GMT-0800
purchase_user_2 = set(events[events["event"] == "8"].user_id_hash.unique())
labels_november_2 = []

for user in list(events.user_id_hash.unique()):
    if user in purchase_user_2:
        labels_november_2.append(1)
    else:
        labels_november_2.append(0)

users_train["purchased2"] = pd.DataFrame(labels_november_2)

users_train.head()

,user_id_hash,purchased,purchased2
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,1
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0


In [18]:
november_event.head()

,session_id,event,event_timestamp,event_value,user_id_hash
249,2232039880727664337,45,1542272300029,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
250,2232039880727664337,45,1542272337049,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
251,2232039880727664337,45,1542272409831,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
252,2232039880727664337,45,1542272491561,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
253,2232039880727664337,45,1542272521397,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


Feature List:
1. total event value
2. number of purchase
3. whether or not purchase
4. total count of different events in event
5. median, last event time difference to our cutoff time
6. 

In [19]:
# 
nov_e2 = november_event[november_event["event"]=="8"][["user_id_hash", "event_value"]].\
                groupby("user_id_hash").sum().reset_index()

In [20]:
event_list = events['event'].value_counts()[:50].index

In [21]:
event_list

CategoricalIndex(['45', '1', '5', '6', '14', '4', '40', '7', '41', '3', '42',
                  '.a5027911885258752', '.a5400102822346752', '44',
                  '.a5516611293544448', '0', '63', '47', '57', '8', '55',
                  '.m5295687445250048', '.a5061295285075968', '50', '9',
                  '.m6311698772393984', '11', '64', '.a5686782234722304', '54',
                  '.m5295687445250048 Accept', '.a5008055943430144',
                  '.m6311588362059776', '.m5100869650219008',
                  '.m6335456823869440', '.a5933275901722624', '43',
                  '.m5295687445250048 Held Back', '49', '.m5925072799399936',
                  '48', '56', '52', '59', '60', '.m6311698772393984 Accept',
                  '58', '.m5100869650219008 Accept',
                  '.m6311588362059776 Accept', '.m4609541589565440'],
                 categories=['.a5008055943430144', '.a5027911885258752', '.a5061295285075968', '.a5400102822346752', '.a5516611293544448', '.a55232742

In [22]:
def features_event(val_raw,val_user,cut_time):
    
    # feature 1: event value
    eventvalue = val_raw[val_raw["event"]=="8"][["user_id_hash", "event_value"]].\
                groupby("user_id_hash").sum().reset_index()
    result = val_user.join(eventvalue.set_index("user_id_hash"), on="user_id_hash", how="left")
    
    # feature 2: number of purchases
    purchase_times = val_raw[val_raw["event"]=="8"].groupby("user_id_hash")\
                    .size().reset_index(name='purchase_counts')
    result = result.join(purchase_times.set_index("user_id_hash"), on="user_id_hash", how="left")
    
    # feature 3: whether or not purchased before
    user_purd = set(november_event[november_event['event']=='8'].user_id_hash)
    result['past_buy'] = [1 if uid in user_purd else 0 for uid in result['user_id_hash']]
    
    # feature 4: total count of different events in event: top 50 events
    event_list = ['45', '1', '5', '6', '14', '4', '40', '7', '41', '3', '42',
                  '.a5027911885258752', '.a5400102822346752', '44',
                  '.a5516611293544448', '0', '63', '47', '57', '8', '55',
                  '.m5295687445250048', '.a5061295285075968', '50', '9',
                  '.m6311698772393984', '11', '64', '.a5686782234722304', '54',
                  '.m5295687445250048 Accept', '.a5008055943430144',
                  '.m6311588362059776', '.m5100869650219008',
                  '.m6335456823869440', '.a5933275901722624', '43',
                  '.m5295687445250048 Held Back', '49', '.m5925072799399936',
                  '48', '56', '52', '59', '60', '.m6311698772393984 Accept',
                  '58', '.m5100869650219008 Accept',
                  '.m6311588362059776 Accept', '.m4609541589565440']
        
    for e in event_list:
        event_tmp = val_raw[val_raw["event"]==e].groupby("user_id_hash").size().reset_index(name=f'event{e}_count')
        result = result.join(event_tmp.set_index("user_id_hash"), on="user_id_hash", how="left")
    
    # feature 5: median, last event time difference to cutoff time
    first_last = val_raw.groupby("user_id_hash")['event_timestamp'].agg(['median','last']).reset_index()
    first_last['median_diff'] = cut_time - first_last['median']
    first_last['last_diff'] = cut_time - first_last['last']
    first_last['user_id_hash'] = first_last['user_id_hash'].astype(str)
    result = result.join(first_last[['user_id_hash','median_diff','last_diff']].set_index("user_id_hash"),on="user_id_hash", how="left")
    
    # feature 6: total previous time spent captured by sessions
    
    
    return result

In [23]:
### Feature for validation/training events

In [24]:
val_raw = events[(events["event_timestamp"] >= 1539586800000) &  # Mon Oct 15 2018 00:00:00 GMT-0700
                   (events["event_timestamp"] < 1543651199000)] # Fri Nov 30 2018 23:59:59 GMT-0800
val_feature = features_event(val_raw,users_val,1543651199000)
val_feature = val_feature.apply(lambda x: x.fillna(0))

In [25]:
train_raw = events[(events["event_timestamp"] < 1542268800000)] # Thu Nov 15 2018 00:00:00 GMT-0800
train_feature = features_event(train_raw,users_train,1542268800000)
train_feature = train_feature.apply(lambda x: x.fillna(0))

In [26]:
train_feature.head()

,user_id_hash,purchased,purchased2,event_value,purchase_counts,past_buy,event45_count,event1_count,event5_count,event6_count,...,event52_count,event59_count,event60_count,event.m6311698772393984 Accept_count,event58_count,event.m5100869650219008 Accept_count,event.m6311588362059776 Accept_count,event.m4609541589565440_count,median_diff,last_diff
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,1,3.492188,1,0,368,12,22,21,...,0,0,0,0,0,0,0,0,2.162291e+09,3.265969e+08
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000,0,0,54,2,3,3,...,0,0,0,0,0,0,0,0,3.815455e+08,3.398837e+08
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000,0,0,22,0,1,0,...,0,0,0,0,0,0,0,0,2.438209e+09,2.437954e+09
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000e+00,0.000000e+00
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,2.045302e+09,1.902918e+09


In [27]:
train_feature.columns.difference(['purchased','purchased2','user_id_hash'])

Index(['event.a5008055943430144_count', 'event.a5027911885258752_count',
       'event.a5061295285075968_count', 'event.a5400102822346752_count',
       'event.a5516611293544448_count', 'event.a5686782234722304_count',
       'event.a5933275901722624_count', 'event.m4609541589565440_count',
       'event.m5100869650219008 Accept_count', 'event.m5100869650219008_count',
       'event.m5295687445250048 Accept_count',
       'event.m5295687445250048 Held Back_count',
       'event.m5295687445250048_count', 'event.m5925072799399936_count',
       'event.m6311588362059776 Accept_count', 'event.m6311588362059776_count',
       'event.m6311698772393984 Accept_count', 'event.m6311698772393984_count',
       'event.m6335456823869440_count', 'event0_count', 'event11_count',
       'event14_count', 'event1_count', 'event3_count', 'event40_count',
       'event41_count', 'event42_count', 'event43_count', 'event44_count',
       'event45_count', 'event47_count', 'event48_count', 'event49_count',
  

In [28]:
X_train = train_feature[train_feature.columns.difference(['purchased','purchased2','user_id_hash'])]
y_train_1 = train_feature[["purchased"]]
y_train_2 = train_feature[["purchased2"]]

In [29]:
X_val = val_feature[val_feature.columns.difference(["purchased1","purchased2",'user_id_hash'])]
y_val_1 = val_feature[['purchased']]
y_val_2 = val_feature[['purchased2']]

In [30]:
X_val.shape

(620988, 56)

### Sessions Features

IDEAS for features:
- Number of sessions
- If this is the first session
- Country of origin
- OS system
-

#### Sessions data frame

In [31]:
sessions.head()

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash
0,5558845121177764917,1542215364580,Asia/Manila,28800000.0,25837591,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,18781111175537580,1539215568666,Asia/Manila,28800000.0,11343848,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,1477540082628742048,1540120743010,Asia/Manila,28800000.0,13499724,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,8184875317380844086,1542671625528,Asia/Manila,28800000.0,32788010,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4706180700083856343,1538997913013,Asia/Manila,28800000.0,5872534,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,4,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [32]:
sessions['user_id_hash'].nunique()

619519

In [33]:
events['user_id_hash'].nunique()

620988

In [34]:
# get unique user list 
users_ses = pd.DataFrame(list(sessions['user_id_hash'].unique()))
users_ses.columns = ["user_id_hash"]

In [35]:
# Validation labels

december_event = events[(events["event_timestamp"] >= 1543651199000) & #  >= Fri Nov 30 2018 23:59:59 GMT-0800
                  (events["event_timestamp"] < 1544255999000)] #  < Fri Dec 07 2018 23:59:59 GMT-0800

# Users who made a purchase in December
purchase_user = set(december_event[december_event["event"] == "8"].user_id_hash.unique()) 

# For validation, the users include all unique users in sessions table
users_val_ses = users_ses.copy()


labels_dec_ses= []
for user in list(sessions['user_id_hash'].unique()):
    if user in purchase_user:
        labels_dec_ses.append(1)
    else:
        labels_dec_ses.append(0)
# label and add to val
users_val_ses["purchased"] = pd.DataFrame(labels_dec_ses)

In [36]:
users_val_ses.shape

(619519, 2)

In [37]:
# second validation label

december_event_2 = events[(events["event_timestamp"] >= 1544255999000)] # Beyond 7 Dec 

purchase_user_2 = set(december_event_2[december_event_2["event"] == "8"].user_id_hash.unique())

labels_dec_ses_2 = []
for user in list(sessions['user_id_hash'].unique()):
    if user in purchase_user_2:
        labels_dec_ses_2.append(1)
    else:
        labels_dec_ses_2.append(0)

users_val_ses["purchased2"] = pd.DataFrame(labels_dec_ses_2)


In [38]:
users_val_ses.shape

(619519, 3)

In [39]:
### Training labels

#  Use Nov-15 to Nov-30 to get features for users labeled in December
november_event = events[(events["event_timestamp"] >= 1542265200000)  # from Mon Nov 15 2018 00:00:00 GMT-0700
                          & (events["event_timestamp"] < 1542960000000)] # Fri Nov 23 2018 00:00:00 GMT-0800

purchase_user = set(november_event[november_event["event"] == "8"].user_id_hash.unique())
# Users_train dataframe
users_train_ses = users_ses.copy()
labels_nov_ses = []

for user in list(sessions['user_id_hash'].unique()):
    if user in purchase_user:
        labels_nov_ses.append(1)
    else:
        labels_nov_ses.append(0)

users_train_ses["purchased"] = pd.DataFrame(labels_nov_ses)

# second training label

november_event_2 = events[(events["event_timestamp"] >= 1542268800000) &  #from Thu Nov 15 2018 00:00:00 GMT-0800
                      (events["event_timestamp"] < 1543478400000)] # till Thu Nov 29 2018 00:00:00 GMT-0800
purchase_user_2 = set(events[events["event"] == "8"].user_id_hash.unique())
labels_nov_ses_2 = []

for user in list(sessions['user_id_hash'].unique()):
    if user in purchase_user_2:
        labels_nov_ses_2.append(1)
    else:
        labels_nov_ses_2.append(0)

users_train_ses["purchased2"] = pd.DataFrame(labels_nov_ses_2)

users_train_ses.head()

,user_id_hash,purchased,purchased2
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,1
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0


In [40]:
#Prepare training set
#Limit sessions by start time
train_raw_ses = sessions[(sessions["start_timestamp"] < 1542268800000)] # Thu Nov 15 2018 00:00:00 GMT-0800

# Join with user_ids in train labels
train_raw_ses = train_raw_ses.join(users_train_ses.set_index("user_id_hash"), on="user_id_hash", how="inner")

In [41]:
val_raw_ses = sessions[(sessions["start_timestamp"] >= 1539586800000) &  # Mon Oct 15 2018 00:00:00 GMT-0700
                   (sessions["start_timestamp"] < 1543651199000)] # Fri Nov 30 2018 23:59:59 GMT-0800

val_raw_ses = val_raw_ses.join(users_val_ses.set_index("user_id_hash"), on="user_id_hash", how="inner")

In [42]:
train_raw_ses.shape

(3140997, 19)

In [43]:
train_raw_ses.head()

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash,purchased,purchased2
0,5558845121177764917,1542215364580,Asia/Manila,28800000.0,25837591,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,1
1,18781111175537580,1539215568666,Asia/Manila,28800000.0,11343848,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,1
2,1477540082628742048,1540120743010,Asia/Manila,28800000.0,13499724,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,1
4,4706180700083856343,1538997913013,Asia/Manila,28800000.0,5872534,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,4,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,1
5,1672434284075955951,1540106255109,Asia/Manila,28800000.0,12495345,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,12,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,1


In [44]:
def features_sessions(df_raw,df_user):
    #Feature #1: number of sessions per user
    num_sessions =df_raw.groupby(['user_id_hash'])['session_id'].count().reset_index()
    result = df_user.join(num_sessions.set_index("user_id_hash"), on="user_id_hash", how="left")
    
    # Feature #2: is_first session
    user_first_session = sessions.groupby(['user_id_hash'])['is_user_first_session'].mean().reset_index()
    user_first_session['first_time'] = 0
    user_first_session.loc[user_first_session['is_user_first_session']==1,'first_time']=1
    result = result.join(user_first_session.drop(['is_user_first_session'], axis = 1).set_index("user_id_hash"), on="user_id_hash", how="left")
    
    
    # Feature #3 Country of origin
    sessions_user =sessions.drop([ 'session_id', 'start_timestamp','previous_sessions_duration',\
                                 'user_created_timestamp',  'session_index',\
                                  'latitude', 'longitude', 'is_user_first_session',
                                 'locale', 'city'], axis =1)
    user_country = sessions_user.drop(['os_name', 'device_id', 'region', 'timezone', 'timezone_offset'], axis= 1)
    user_country.drop_duplicates(inplace = True)
    country_one_hot = pd.get_dummies(user_country['country'])
    user_country = pd.concat([user_country, country_one_hot], axis = 1).reset_index()
    
    result = result.join(user_country.drop(['index'], axis = 1).set_index("user_id_hash"), on="user_id_hash", how="left")
    
    # Feature #4 OS System
    user_os = sessions_user.drop(['country', 'device_id', 'region', 'timezone', 'timezone_offset'], axis= 1)
    user_os.drop_duplicates(inplace = True)

    result = result.join(user_os.set_index("user_id_hash"), on="user_id_hash", how="left")
    
    return result
    

In [45]:
train_features_ses = features_sessions(train_raw_ses,users_train_ses)
train_features_ses = train_features_ses.drop(['country', 'os_name'], axis = 1).apply(lambda x: x.fillna(0))

In [46]:
val_features_ses= features_sessions(val_raw_ses,users_val_ses)
#val_feature = val_feature.apply(lambda x: x.fillna(0))

In [47]:
val_features_ses = val_features_ses.drop(['country', 'os_name'], axis = 1).apply(lambda x: x.fillna(0))

In [48]:
train_feature.shape

(620988, 58)

In [49]:
train_features_ses.shape

(626113, 233)

In [50]:
# Join event features and sessions features
train_feat = train_feature.join(train_features_ses.drop(['purchased', 'purchased2'], axis = 1).set_index("user_id_hash"), on="user_id_hash", how="left")



In [51]:
val_feat = val_feature.join(val_features_ses.drop(['purchased', 'purchased2'], axis = 1).set_index("user_id_hash"), on="user_id_hash", how="left")



In [52]:
X_train = train_feat[train_feat.columns.difference(['purchased','purchased2','user_id_hash'])]
X_val = val_feat[val_feat.columns.difference(['purchased','purchased2','user_id_hash'])]

In [53]:
X_train.head()

,AD,AE,AF,AG,AI,AL,AM,AO,AR,AS,...,event7_count,event8_count,event9_count,event_value,first_time,last_diff,median_diff,past_buy,purchase_counts,session_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8,1,0,3.492188,0.0,3.265969e+08,2.162291e+09,0,1,31.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,0,0,0.000000,0.0,3.398837e+08,3.815455e+08,0,0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,0,0,0.000000,1.0,2.437954e+09,2.438209e+09,0,0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.000000,1.0,0.000000e+00,0.000000e+00,0,0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.000000,1.0,1.902918e+09,2.045302e+09,0,0,1.0


In [54]:
y_train_1 = train_feat[["purchased"]]
y_train_2 = train_feat[["purchased2"]]

In [55]:
y_val_1 = val_feat[['purchased']]
y_val_2 = val_feat[['purchased2']]

## train with LGB model

In [57]:
# loop for random search
n_iterations=10

print ("Random search start...")

for col in ['model1_7day','model2_14day']:
    print(f"* training {col} ")
    #y = target[col]
    roc_auc_mean = []
    dict_list = []
    
    for i in range(0, n_iterations):

        param_dist = {'n_estimators' : choice([250,300,350,400,450]),
                  'bagging_fraction': choice([0.5, 0.7, 0.8, 0.9]),
                  'learning_rate': choice([0.05, 0.1, 0.3, 0.5]),
                  'is_unbalance': True,
                  'max_bin': choice([3, 5, 10, 15, 18, 20, 25]),
                  'boosting_type' : choice(['gbdt', 'dart']),
                  'max_depth': choice([2,3,4,5]),      
                  'feature_fraction': choice([0.7, 0.8, 0.9]),
                  'lambda_l1': choice([0, 10, 20, 30, 40]),
                  'objective': 'binary', 
                  'metric': 'auc'} 

        roc_l = []
        
       # y_train_1,y_train_2,X_train
       # y_val_1,y_val_2,X_val

        #for train, test in kf.split(X_tf,y):

        # training
        if col == 'model1_7day':
            y_train = y_train_1
            y_val = y_val_1
        else: 
            y_train = y_train_2
            y_val = y_val_2
            
        gbm = lgb.LGBMClassifier(**param_dist)
        gbm.fit(X_train,y_train)
        # predicting
        y_pred = np.round(gbm.predict_proba(X_val)[:,1],3)
        
        roc = roc_auc_score(y_val, y_pred)
        roc_l.append(roc)

        roc_array = np.asarray(roc_l)

        roc_auc_mean.append(roc_array.mean())
        dict_list.append(param_dist)
        #gc.collect()
        

    results_pd = pd.DataFrame({"roc_auc_mean": roc_auc_mean,"parameters": dict_list})    
    results_pd.sort_values("roc_auc_mean", ascending = False, axis = 0, inplace = True)
    
    top_pd = results_pd.head(1)
    
    print(f"--> Best AUC:{top_pd.iloc[0,0]} using {top_pd.iloc[0,1]}")

Random search start...
* training model1_7day 
--> Best AUC:1.0 using {'n_estimators': 350, 'bagging_fraction': 0.9, 'learning_rate': 0.05, 'is_unbalance': True, 'max_bin': 5, 'boosting_type': 'dart', 'max_depth': 5, 'feature_fraction': 0.7, 'lambda_l1': 30, 'objective': 'binary', 'metric': 'auc'}
* training model2_14day 
--> Best AUC:0.9779002521854568 using {'n_estimators': 450, 'bagging_fraction': 0.5, 'learning_rate': 0.3, 'is_unbalance': True, 'max_bin': 5, 'boosting_type': 'dart', 'max_depth': 2, 'feature_fraction': 0.8, 'lambda_l1': 30, 'objective': 'binary', 'metric': 'auc'}


In [85]:
# 
param_dist = {'n_estimators' : 350,
                  'bagging_fraction':  0.9,
                  'learning_rate':0.05,
                  'is_unbalance': True,
                  'max_bin': 5,
                  'boosting_type': 'dart',
                  'max_depth': 5,      
                  'feature_fraction':0.7,
                  'lambda_l1': 30,
                  'objective': 'binary', 
                  'metric': 'auc'} 
t = time.time()
#Model for 7 days
gbm_1 = lgb.LGBMClassifier(**param_dist)
gbm_1.fit(X_train,y_train_1)
# predicting
y_pred_1 = gbm_1.predict_proba(X_val)
score_1  = y_pred_1[:,1]

print(f'auc score = {roc_auc_score(y_val_1,score_1)}')
print(f"Time use:{time.time()-t:.3f}s") 

auc score = 1.0
Time use:25.852s


In [86]:
#Model for 14 days
param_dist_2 = {'n_estimators' : 450,
                  'bagging_fraction':  0.5,
                  'learning_rate':0.3,
                  'is_unbalance': True,
                  'max_bin': 5,
                  'boosting_type': 'dart',
                  'max_depth': 2,      
                  'feature_fraction':0.8,
                  'lambda_l1': 30,
                  'objective': 'binary', 
                  'metric': 'auc'} 
t = time.time()

gbm_2 = lgb.LGBMClassifier(**param_dist_2)
gbm_2.fit(X_train,y_train_2)
# predicting
y_pred_2 = gbm_2.predict_proba(X_val)
score_2 = y_pred_2[:,1]

print(f'auc score = {roc_auc_score(y_val_2,score_2)}')
print(f"Time use:{time.time()-t:.3f}s") 

auc score = 0.9779260704027422
Time use:32.703s


In [87]:
# Preparing test set
sample = pd.read_csv(os.path.expanduser("~/USF/adv_ml/final/sample_submission_2.csv"))


#users = pd.DataFrame(list(events.user_id_hash.unique()))
#users.columns = ["user_id_hash"]
test_raw_ses = sessions[(sessions['start_timestamp'] >= 1541055600000)] # Nov 1
test_raw_event = events[(events["event_timestamp"] >= 1541055600000)] # November 1
user_test = users


In [77]:
# Join with user_ids in test labels
test_raw_ses = test_raw_ses.join(user_test.set_index("user_id_hash"), on="user_id_hash", how="inner")

In [78]:
# Create events feautures
test_feature = features_event(test_raw_event,user_test,1544860800000)
test_feature = test_feature.apply(lambda x: x.fillna(0))


In [81]:
# create sessions features
test_features_ses = features_sessions(test_raw_ses,user_test)
test_features_ses = test_features_ses.drop(['country', 'os_name'], axis = 1).apply(lambda x: x.fillna(0))

In [82]:
# Join event and sessions features
test_feat = test_feature.join(test_features_ses.drop(['purchased', 'purchased2'], axis = 1).set_index("user_id_hash"), on="user_id_hash", how="left")


In [83]:
X_test = test_feat[test_feat.columns.difference(['purchased','purchased2','user_id_hash'])]

In [84]:
X_test.head()

,AD,AE,AF,AG,AI,AL,AM,AO,AR,AS,...,event7_count,event8_count,event9_count,event_value,first_time,last_diff,median_diff,past_buy,purchase_counts,session_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,1,0,3.492188,0,2.918597e+09,2.670147e+09,0,1,33.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,0,0,0.000000,0,2.931884e+09,2.973545e+09,0,0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.000000,1,0.000000e+00,0.000000e+00,0,0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.000000,1,2.337255e+09,2.337667e+09,0,0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.000000,1,0.000000e+00,0.000000e+00,0,0,0.0


In [88]:
pred_1 = gbm_1.predict_proba(X_test)
pred_2 = gbm_2.predict_proba(X_test)

test_feat["user_purchase_binary_7_days"] = pd.DataFrame(pred_1[:,-1])
test_feat["user_purchase_binary_14_days"] = pd.DataFrame(pred_2[:,-1])

In [90]:
submission = sample[['user_id_hash']].merge(test_feat[['user_id_hash','user_purchase_binary_7_days',\
                                             'user_purchase_binary_14_days']],on='user_id_hash',\
                               how='left')
# for users do not have previous data, mark as 0
submission = submission.fillna(0)

In [91]:
submission.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.010551,0.153194
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.001379,0.000908
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.002738,0.035859
3,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.002738,0.035859
4,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.001407,0.000906


In [92]:
submission.to_csv(os.path.expanduser("~/USF/adv_ml/final/submission_masha.csv"),index = False)